In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display
from sqlalchemy import create_engine, Column, Integer, Float, String, Boolean, func, text
from sqlalchemy.orm import declarative_base, sessionmaker

In [ ]:
Base = declarative_base()

class Customer(Base):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    racket = Column(String)
    tension = Column(Integer)
    stringType = Column(String)
    count = Column(Integer)
    completed = Column(Integer)
    owed = Column(Float)
    paid = Column(Boolean)
    def __repr__(self):
        return f'Customer: {self.name}, Racket: {self.racket}, Tension: {self.tension}, StringType: {self.stringType}, Remaining to String: {self.count - self.completed}, Owed: {self.owed}, Paid: {self.paid}'

In [3]:
engine = create_engine('sqlite:///badminton_customer.db')
Base.metadata.create_all(engine)

def ensure_customer_columns(engine):
    with engine.connect() as conn:
        cols = [row[1] for row in conn.execute(text("PRAGMA table_info(customers)")).fetchall()]
        if 'tension' not in cols:
            conn.execute(text("ALTER TABLE customers ADD COLUMN tension INTEGER"))
        if 'stringType' not in cols:
            conn.execute(text("ALTER TABLE customers ADD COLUMN stringType VARCHAR"))
        conn.commit()

ensure_customer_columns(engine)

all_names = []
all_rackets = []

Session = sessionmaker(bind=engine)
session = Session()

name_input_label = widgets.Label("Customer Name:")

name_selection = widgets.Combobox(
    placeholder = 'Customer Name',
    options = list(customer.name for customer in session.query(Customer).all() if customer.name),
    ensure_option = False
)
name_button = widgets.Button(
    description = 'Confirm',
    button_style = 'success'
)

In [4]:
racket_selection = widgets.Combobox(
    placeholder = 'Racket Name',
    options = list(customer.racket for customer in session.query(Customer).all() if customer.racket),
    ensure_option = False
)

add_racket = widgets.Button(
    description = 'Add Racket',
    button_style = 'success'
)

In [5]:
add_button = widgets.Button(
    description = '+1 Racket',
    button_style = 'success'
)

In [6]:
completed_button = widgets.Button(
    description = '+1 Completed',
    button_style = 'success'
)

In [7]:
paid_button = widgets.Button(
    description = 'Paid $20',
    button_style = 'success'
)

In [8]:
name_label = widgets.Label()
racket_label = widgets.Label()
count_label = widgets.Label()
completed_label = widgets.Label()
remaining_label = widgets.Label()
owed_label = widgets.Label()
paid_mark = widgets.Label()

tension_input = widgets.BoundedIntText(value=0, min=0, max=80, description='Tension')
string_type_input = widgets.Text(value='', description='String Type', placeholder='e.g., BG65 / Aerobite')
entries_table_box = VBox([])


In [9]:
current = {'customer': None}

In [10]:
def refresh_options():
    global all_names, all_rackets
    all_names = [c[0] for c in session.query(Customer.name).filter(Customer.name.isnot(None)).all() if c[0]]
    name_selection.options = list(sorted(set(all_names)))

    all_rackets = [r[0] for r in session.query(Customer.racket).filter(Customer.racket.isnot(None)).all() if r[0]]
    racket_selection.options = list(sorted(set(all_rackets)))


In [11]:
def show_customer(customer):
    # customer is the currently selected name 'profile' row
    if not customer or not customer.name:
        entries_table_box.children = tuple()
        return

    name_key = customer.name.lower()
    entries = (
        session.query(Customer)
        .filter(func.lower(Customer.name) == name_key)
        .filter(Customer.racket.isnot(None))
        .filter(Customer.racket != '')
        .order_by(Customer.id.asc())
        .all()
    )

    # Basic labels
    name_label.value = f'Name: {customer.name.title() if customer.name else ""}'
    racket_label.value = ''
    count_label.value = f'Total Entries: {len(entries)}'
    completed_label.value = ''
    remaining_label.value = ''

    unpaid_count = sum(1 for e in entries if not bool(e.paid))
    total_owed = float(unpaid_count * 20)
    owed_label.value = f'Total Owed: ${total_owed}'
    paid_mark.value = 'Paid: ✅' if total_owed == 0.0 else 'Paid: ❌'

    header = HBox([
        widgets.HTML('<b>Name</b>'),
        widgets.HTML('<b>Racket</b>'),
        widgets.HTML('<b>Tension</b>'),
        widgets.HTML('<b>String Type</b>'),
        widgets.HTML('<b>Paid</b>'),
        widgets.HTML('<b>Delete</b>'),
    ])

    rows = [header]

    for row in entries:
        paid_cb = widgets.Checkbox(value=bool(row.paid), indent=False)
        del_btn = widgets.Button(description='Delete', button_style='danger')

        def make_on_paid(entry_id):
            def _on_change(change):
                if change.get('name') != 'value':
                    return
                rec = session.query(Customer).filter_by(id=entry_id).first()
                if not rec:
                    return
                rec.paid = bool(change.get('new'))
                session.commit()
                show_customer(customer)
            return _on_change

        def make_on_delete(entry_id):
            def _on_click(_b):
                rec = session.query(Customer).filter_by(id=entry_id).first()
                if not rec:
                    return
                session.delete(rec)
                session.commit()
                refresh_options()
                show_customer(customer)
            return _on_click

        paid_cb.observe(make_on_paid(row.id), names='value')
        del_btn.on_click(make_on_delete(row.id))

        rows.append(HBox([
            widgets.Label(row.name or '', layout=widgets.Layout(width='140px')),
            widgets.Label(row.racket or '', layout=widgets.Layout(width='180px')),
            widgets.Label('' if row.tension is None else str(row.tension), layout=widgets.Layout(width='80px')),
            widgets.Label(row.stringType or '', layout=widgets.Layout(width='180px')),
            paid_cb,
            del_btn,
        ]))

    entries_table_box.children = tuple(rows)


In [12]:
details_box = VBox([
    HBox([racket_selection, tension_input, string_type_input, add_racket]),
    HBox([add_button, completed_button, paid_button]),
    VBox([name_label, racket_label, count_label, completed_label, remaining_label, owed_label, paid_mark]),
    entries_table_box,
])
name_button.layout.display = "none"
name_row = HBox([name_input_label, name_selection, name_button])

root = VBox([name_row])


In [13]:
def on_name_typing(change):
    typed = change['new'] or ''
    # Don't strip here - preserve what user is typing for dropdown
    if typed:
        typed_lower = typed.lower()
        # Case-insensitive filtering - prioritize matches that start with typed text
        starts_with = [n for n in all_names if n.lower().startswith(typed_lower)]
        contains = [n for n in all_names if typed_lower in n.lower() and n not in starts_with]
        # Combine: starts_with first, then contains, then the typed value
        filtered = starts_with + contains
        # Always include the typed value so combobox can accept it
        if typed not in filtered:
            filtered.append(typed)
        name_selection.options = list(filtered)  # Use list instead of tuple
    else:
        name_selection.options = list(sorted(set(all_names)))  # Use list instead of tuple
    
    if current['customer'] is None:
        name_button.layout.display = "none" if typed == "" else "inline-flex"


In [14]:
def on_racket_typing(change):
    typed = change['new'] or ''
    # Don't strip here - preserve what user is typing for dropdown
    if typed:
        typed_lower = typed.lower()
        # Case-insensitive filtering - prioritize matches that start with typed text
        starts_with = [r for r in all_rackets if r.lower().startswith(typed_lower)]
        contains = [r for r in all_rackets if typed_lower in r.lower() and r not in starts_with]
        # Combine: starts_with first, then contains, then the typed value
        filtered = starts_with + contains
        # Always include the typed value so combobox can accept it
        if typed not in filtered:
            filtered.append(typed)
        racket_selection.options = list(filtered)  # Use list instead of tuple
    else:
        racket_selection.options = list(sorted(set(all_rackets)))  # Use list instead of tuple

In [15]:
def confirm_name(b):
    name = name_selection.value.strip()
    if not name:
        return

    name_title = name.title()
    existing = session.query(Customer).filter(func.lower(Customer.name) == name.lower()).first()

    if not existing:
        # Create a 'profile' row (no racket). Entries are separate rows with racket set.
        existing = Customer(
            name=name_title,
            racket=None,
            tension=None,
            stringType=None,
            count=0,
            completed=0,
            owed=0.0,
            paid=False,
        )
        session.add(existing)
        session.commit()
        refresh_options()
    else:
        # normalize stored casing
        if existing.name != name_title:
            existing.name = name_title
            session.commit()
            refresh_options()

    current['customer'] = existing
    show_customer(existing)
    if details_box not in root.children:
        root.children = (name_row, details_box)


In [16]:
def on_add_racket(b):
    current['customer'].count += 1
    current['customer'].owed += 20
    session.commit()
    show_customer(current['customer'])

In [17]:
def on_completed_racket(b):
    count = current['customer'].count or 0
    current_completed = current['customer'].completed or 0
    # Ensure completed doesn't exceed count
    current['customer'].completed = max(0, min(count, current_completed + 1))
    session.commit()
    show_customer(current['customer'])

In [18]:
def on_racket_change(b):
    if current['customer'] is None or not current['customer'].name:
        return

    racket = racket_selection.value.strip()
    if not racket:
        return

    tension = tension_input.value
    string_type = (string_type_input.value or '').strip()

    racket_title = racket.title()

    # Enforce uniqueness: (name, racket, tension, stringType)
    exists = (
        session.query(Customer)
        .filter(func.lower(Customer.name) == current['customer'].name.lower())
        .filter(func.lower(Customer.racket) == racket_title.lower())
        .filter(Customer.tension == tension)
        .filter(func.coalesce(Customer.stringType, '') == string_type)
        .first()
    )
    if exists:
        # Duplicate - do nothing
        racket_selection.value = ''
        return

    entry = Customer(
        name=current['customer'].name,
        racket=racket_title,
        tension=tension,
        stringType=string_type,
        count=1,
        completed=0,
        owed=0.0,
        paid=False,
    )
    session.add(entry)
    session.commit()

    refresh_options()
    show_customer(current['customer'])

    racket_selection.value = ''
    tension_input.value = 0
    string_type_input.value = ''


In [19]:
def on_paid(b):
    current['customer'].owed = max(current['customer'].owed - 20, 0.0)
    current['customer'].paid = current['customer'].owed == 0.0
    session.commit()
    show_customer(current['customer'])

In [20]:
name_selection.observe(on_name_typing, names ='value')
name_button.on_click(confirm_name)
racket_selection.observe(on_racket_typing, names='value')
add_racket.on_click(on_racket_change)
add_button.on_click(on_add_racket)
completed_button.on_click(on_completed_racket)
paid_button.on_click(on_paid)

In [21]:
refresh_options()
display(root)

In [22]:
# import ipywidgets as widgets
# from ipywidgets import HBox, VBox
# from IPython.display import display
# from sqlalchemy import create_engine, Column, Integer, Float, String, Boolean, func, inspect
# from sqlalchemy.orm import declarative_base, sessionmaker
# import os
#
# # -------------------- DB SETUP --------------------
# Base = declarative_base()
#
# class Customer(Base):
#     __tablename__ = 'customers'
#     id = Column(Integer, primary_key=True)
#     name = Column(String)
#     racket = Column(String)
#     count = Column(Integer)
#     completed = Column(Integer)
#     owed = Column(Float)
#     paid = Column(Boolean)
#
#     def __repr__(self):
#         return f'Customer: {self.name}, Racket: {self.racket}, Remaining to String: {self.count - self.completed}, Owed: {self.owed}, Paid: {self.paid}'
#
# # Database file path
# db_path = 'badminton_customer.db'
# db_url = f'sqlite:///{db_path}'
#
# print("=" * 50)
# print("DATABASE CONNECTION TEST")
# print("=" * 50)
#
# # Check if database file exists
# if os.path.exists(db_path):
#     file_size = os.path.getsize(db_path)
#     print(f"✅ Database file exists: {db_path} ({file_size} bytes)")
# else:
#     print(f"⚠️  Database file does not exist yet: {db_path}")
#     print("   It will be created when we create the tables.")
#
# # Create engine and test connection
# try:
#     engine = create_engine(db_url, echo=False)
#     print(f"✅ Engine created: {db_url}")
#
#     # Test connection
#     with engine.connect() as conn:
#         print("✅ Database connection successful!")
#
#     # Create tables
#     Base.metadata.create_all(engine)
#     print("✅ Tables created/verified")
#
#     # Check if customers table exists
#     inspector = inspect(engine)
#     tables = inspector.get_table_names()
#     print(f"✅ Tables in database: {tables}")
#
#     if 'customers' in tables:
#         columns = [col['name'] for col in inspector.get_columns('customers')]
#         print(f"✅ Customers table columns: {columns}")
#
# except Exception as e:
#     print(f"❌ Database error: {e}")
#     import traceback
#     traceback.print_exc()
#     raise
#
# # Create session
# try:
#     Session = sessionmaker(bind=engine)
#     session = Session()
#     print("✅ Session created")
#
#     # Test query
#     customer_count = session.query(Customer).count()
#     print(f"✅ Database query successful! Found {customer_count} customers")
#
#     if customer_count > 0:
#         print("\nExisting customers:")
#         for customer in session.query(Customer).all():
#             print(f"  - ID: {customer.id}, Name: {customer.name}, Racket: {customer.racket}")
#     else:
#         print("   (No customers in database yet)")
#
# except Exception as e:
#     print(f"❌ Session/Query error: {e}")
#     import traceback
#     traceback.print_exc()
#     raise
#
# print("=" * 50)
# print()
#
# all_names = []
# all_rackets = []
#
# # -------------------- WIDGETS --------------------
# name_input_label = widgets.Label("Customer Name:")
#
# name_selection = widgets.Combobox(
#     placeholder='Customer Name',
#     options=tuple(),
#     ensure_option=False,
#     layout=widgets.Layout(width='300px')
# )
#
# name_button = widgets.Button(
#     description='Confirm',
#     button_style='success'
# )
#
# racket_selection = widgets.Combobox(
#     placeholder='Racket Name',
#     options=tuple(),
#     ensure_option=False,
#     layout=widgets.Layout(width='300px')
# )
#
# add_racket = widgets.Button(
#     description='Add Racket',
#     button_style='success'
# )
#
# add_button = widgets.Button(
#     description='+1 Racket',
#     button_style='success'
# )
#
# completed_button = widgets.Button(
#     description='+1 Completed',
#     button_style='success'
# )
#
# paid_button = widgets.Button(
#     description='Paid $20',
#     button_style='success'
# )
#
# name_label = widgets.Label()
# racket_label = widgets.Label()
# count_label = widgets.Label()
# completed_label = widgets.Label()
# remaining_label = widgets.Label()
# owed_label = widgets.Label()
# paid_mark = widgets.Label()
#
# # Output widget for displaying messages
# output = widgets.Output()
#
# current = {'customer': None}
#
# # -------------------- HELPER FUNCTIONS --------------------
# def refresh_options():
#     global all_names, all_rackets
#     try:
#         session.expire_all()
#         customers = session.query(Customer).all()
#         print(f"🔍 refresh_options: Found {len(customers)} customers in database")
#
#         all_names = [c.name for c in customers if c.name]
#         name_selection.options = tuple(sorted(set(all_names)))
#         print(f"🔍 refresh_options: Updated name options: {len(all_names)} names")
#
#         all_rackets = []
#         for customer in customers:
#             if customer.racket:
#                 all_rackets.extend([r.strip() for r in customer.racket.split(', ') if r.strip()])
#         all_rackets = sorted(set(all_rackets))
#         racket_selection.options = tuple(all_rackets)
#         print(f"🔍 refresh_options: Updated racket options: {len(all_rackets)} rackets")
#     except Exception as e:
#         print(f"❌ Error in refresh_options: {e}")
#         import traceback
#         traceback.print_exc()
#
# def show_customer(customer):
#     name_label.value = f'Name: {customer.name}'
#     racket_label.value = f'Racket: {customer.racket or ""}'
#     count_label.value = f'Total Rackets: {customer.count or 0}'
#     completed_label.value = f'Completed Rackets: {customer.completed or 0}'
#     remaining_label.value = f'Remaining to String: {(customer.count or 0) - (customer.completed or 0)}'
#     owed_label.value = f'Amount Owed: ${customer.owed or 0.0:.2f}'
#     paid_mark.value = '✅' if (customer.paid or (customer.owed or 0.0) == 0.0) else '❌'
#
# # -------------------- LAYOUT --------------------
# # Always include the button in the layout, just show/hide it
# name_button.layout.display = "none"  # Hidden initially
# name_row = HBox([name_input_label, name_selection, name_button])
#
# details_box = VBox([
#     HBox([racket_selection, add_racket]),
#     HBox([add_button, completed_button, paid_button]),
#     VBox([name_label, racket_label, count_label, completed_label, remaining_label, owed_label, paid_mark])
# ])
#
# root = VBox([name_row, output])
#
# # -------------------- CALLBACKS --------------------
# def on_name_typing(change):
#     typed = (change['new'] or '').strip()
#     typed_lower = typed.lower()
#
#     # Filter options as user types, but always include the current typed value
#     if typed:
#         filtered = [n for n in all_names if typed_lower in n.lower()]
#         # Always include the current typed value if it's not already in the list
#         if typed not in filtered:
#             filtered.append(typed)
#         name_selection.options = tuple(sorted(set(filtered)))
#     else:
#         name_selection.options = tuple(sorted(set(all_names)))
#
#     # Show/hide confirm button based on whether name is typed
#     if current['customer'] is None:
#         name_button.layout.display = "none" if typed == '' else "inline-flex"
#
# def confirm_name(b):
#     # Use regular print statements - they're always visible in notebook
#     try:
#         print("🔵 Confirm button clicked!")
#         name = name_selection.value
#         print(f"Raw name value: {repr(name)}")
#
#         if name is None:
#             print("❌ Name is None, returning")
#             return
#
#         name = name.strip()
#         if not name:
#             print("❌ Name is empty after strip, returning")
#             return
#
#         print(f"✅ Processing name: {name}")
#
#         # Ensure the typed name is in the options (for Combobox to accept it)
#         current_options = list(name_selection.options)
#         if name not in current_options:
#             current_options.append(name)
#             name_selection.options = tuple(sorted(set(current_options)))
#
#         # Test database query first
#         print("🔍 Testing database query...")
#         test_count = session.query(Customer).count()
#         print(f"🔍 Database has {test_count} total customers")
#
#         # Case-insensitive search to prevent duplicates
#         print(f"🔍 Searching for customer with name: '{name}'")
#         existing = session.query(Customer).filter(func.lower(Customer.name) == name.lower()).first()
#
#         if not existing:
#             print(f"➕ Creating new customer: {name}")
#             new_customer = Customer(
#                 name=name,
#                 racket='',
#                 count=0,
#                 completed=0,
#                 owed=0.0,
#                 paid=False,
#             )
#             print(f"🔍 Customer object created: {new_customer}")
#             session.add(new_customer)
#             print("🔍 Customer added to session")
#             session.commit()
#             print("🔍 Changes committed to database")
#             session.refresh(new_customer)
#             print(f"🔍 Customer refreshed, ID: {new_customer.id}")
#
#             # Verify it was saved
#             verify = session.query(Customer).filter_by(id=new_customer.id).first()
#             if verify:
#                 print(f"✅ Verified: Customer saved to database - ID: {verify.id}, Name: {verify.name}")
#             else:
#                 print("❌ ERROR: Customer not found in database after commit!")
#
#             refresh_options()
#             current['customer'] = new_customer
#             print(f"✅ New customer created with ID: {new_customer.id}")
#         else:
#             print(f"📋 Found existing customer: {name} (ID: {existing.id})")
#             current['customer'] = existing
#
#         # Always show the customer
#         show_customer(current['customer'])
#         print("✅ Customer details updated in UI")
#
#         # Show details box if not already shown - FORCE IT TO SHOW
#         current_children = list(root.children)
#         if details_box not in current_children:
#             # Keep name_row and output, add details_box
#             root.children = tuple([name_row, output, details_box])
#             print("✅ Details box added to UI")
#         else:
#             print("✅ Details box already visible")
#
#         print("✅ Customer displayed successfully!")
#
#     except Exception as e:
#         error_msg = f"❌ Error in confirm_name: {e}"
#         print(error_msg)
#         import traceback
#         traceback.print_exc()
#
# def on_add_racket(b):
#     if current['customer'] is None:
#         return
#     current['customer'].count = (current['customer'].count or 0) + 1
#     current['customer'].owed = (current['customer'].owed or 0.0) + 20
#     current['customer'].paid = False
#     session.commit()
#     show_customer(current['customer'])
#
# def on_completed_racket(b):
#     if current['customer'] is None:
#         return
#     current['customer'].completed = (current['customer'].completed or 0) + 1
#     session.commit()
#     show_customer(current['customer'])
#
# def on_racket_typing(change):
#     typed = (change['new'] or '').strip()
#     typed_lower = typed.lower()
#     # Filter options as user types, but always include the current typed value
#     if typed:
#         filtered = [r for r in all_rackets if typed_lower in r.lower()]
#         # Always include the current typed value if it's not already in the list
#         if typed not in filtered:
#             filtered.append(typed)
#         racket_selection.options = tuple(sorted(set(filtered)))
#     else:
#         racket_selection.options = tuple(all_rackets)
#
# def on_racket_change(b):
#     if current['customer'] is None:
#         return
#     racket = racket_selection.value
#     if racket:
#         racket = racket.strip()
#     if not racket:
#         return
#
#     old = current['customer'].racket
#     if old:
#         old = old.strip()
#     rackets = [] if not old or old == '' else old.split(', ')
#     if racket not in rackets:
#         rackets.append(racket)
#     current['customer'].racket = ', '.join(rackets)
#     session.commit()
#     refresh_options()
#     show_customer(current['customer'])
#     racket_selection.value = ""
#
# def on_paid(b):
#     if current['customer'] is None:
#         return
#     current['customer'].owed = max((current['customer'].owed or 0.0) - 20, 0.0)
#     current['customer'].paid = (current['customer'].owed == 0.0)
#     session.commit()
#     show_customer(current['customer'])
#
# # -------------------- WIRE UP EVENTS --------------------
# # Test function to verify button works
# def test_button(b):
#     print("🧪 TEST: Button was clicked!")
#
# name_selection.observe(on_name_typing, names='value')
# name_button.on_click(confirm_name)
# # Also add test handler to verify button works
# name_button.on_click(test_button)
# racket_selection.observe(on_racket_typing, names='value')
# add_racket.on_click(on_racket_change)
# add_button.on_click(on_add_racket)
# completed_button.on_click(on_completed_racket)
# paid_button.on_click(on_paid)
#
# # Verify button is connected
# print(f"🔧 Button click handlers count: {len(name_button._click_handlers.callbacks) if hasattr(name_button._click_handlers, 'callbacks') else 'N/A'}")
# print(f"🔧 Button description: {name_button.description}")
# print(f"🔧 Button visible: {name_button.layout.display}")
#
# # -------------------- START --------------------
# print("\n" + "=" * 50)
# print("INITIALIZING UI")
# print("=" * 50)
# refresh_options()
# print("✅ UI initialized")
# print("=" * 50 + "\n")
#
# display(root)